# Initialization

Test notebook for the DataHandlers. Test the CMAPSS DataHandler

First we import the necessary packages and create the global variables.

In [1]:
import numpy as np
import math
import tensorflow as tf
from keras.layers.core import Activation

import csv

from time import time
from datetime import datetime
from data_handler_CMAPS import CMAPSDataHandler
from tunable_model import SequenceTunableModelRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D, LSTM, CuDNNLSTM
from keras.optimizers import Adam, SGD
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

import CMAPSAuxFunctions
from CMAPSAuxFunctions import TrainValTensorBoard

from numpy.random import seed
from tensorflow import set_random_seed

Using TensorFlow backend.


# Define different types of Keras models

In [2]:
dropout_ratio = 0.2

def RULmodel_LSTM(input_shape):
    """Define the RNN model"""
    
    #Create a sequential model
    model = Sequential()
#     model.add(CuDNNLSTM(input_shape=input_shape, units=32, return_sequences=True, name='lstm1'))
#     model.add(Dropout(dropout_ratio))
#     model.add(CuDNNLSTM(units=32, return_sequences=False, name='lstm2'))
#     model.add(Dropout(dropout_ratio))
#     model.add(Dense(16, activation='relu', kernel_initializer='glorot_normal',name='fc1'))
#     model.add(Dropout(dropout_ratio))
#     model.add(Dense(8, activation='relu', kernel_initializer='glorot_normal', name='fc2'))
#     model.add(Dropout(dropout_ratio))
#     model.add(Dense(1, activation='linear', kernel_initializer='glorot_normal', name='out'))
#     model.add(CuDNNLSTM(input_shape=input_shape, units=150, return_sequences=False, name='lstm1'))
#     model.add(LSTM(input_shape=input_shape,units=100,return_sequences=True,name='lstm1'))
#     model.add(Dropout(dropout_ratio))
#     model.add(LSTM(units=50,  return_sequences=False,name='lstm2'))
#     model.add(Dropout(dropout_ratio))
#     model.add(Dense(1, activation='linear', kernel_initializer='glorot_normal', name='out'))
    

    model.add(LSTM(
         input_shape=(input_shape),
         units=100,
         return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(
          units=50,
          return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))
    model.add(Activation("linear"))
    
   
    
    return model

# Create the Data Handler

In [3]:
#Selected as per CNN paper
features = ['Op. Settings 1', 'Op. Settings 2', 'Op. Settings 3','Cycle','T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([1,2, 3, 4,5,6 ,7, 8, 9, 10, 11, 12, 13, 14, 15,16, 17, 18, 19,20, 21,22,23,24,25])
selected_features = list(features[i] for i in selected_indices-1)
print(selected_features)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 125

dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)
#dHandler_cmaps.load_data(verbose=1, cross_validation_ratio=0.2)

['Op. Settings 1', 'Op. Settings 2', 'Op. Settings 3', 'Cycle', 'T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']


# Create Tunable Model

In [4]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
scaler  = StandardScaler()

In [5]:
def get_compiled_model(shape, model_type='lstm'):
    
    K.clear_session()  #Clear the previous tensorflow graph
    
    #To test the model without randomness

    
    #Shared parameters for the models
    optimizer = Adam(lr=0.001,beta_1=0.5)#beta_1=0.5#'rmsprop'#Adam(lr=0.001)#beta_1=0.5
    
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    model = None

    #Create and compile the models

    model = RULmodel_LSTM(shape)
    model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    
    print(model.summary())

    return model

In [6]:
num_features = len(selected_features)

shape = (window_size, num_features)
model = get_compiled_model(shape, model_type='lstm')
tModel = SequenceTunableModelRegression('ModelRUL_LSTM_1', model, lib_type='keras', data_handler=dHandler_cmaps)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 30, 100)           50400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 100)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
_________________________________________________________________
activation_1 (Activation)    (None, 1)                 0         
Total params: 80,651
Trainable params: 80,651
Non-trainable params: 0
_________________________________________________________________
None


# Load Data

In [7]:
tModel.data_handler.sequence_length = 30
#tModel.data_handler.sequence_length = maxWindowSize[datasetNumber]
tModel.data_handler.sequence_stride = 1
tModel.data_handler.max_rul = 125

In [8]:
# #Load sequenced data (do not unroll sequence into a single feature vector)

tModel.data_handler.data_scaler = min_max_scaler
#tModel.data_handler.data_scaler = scaler
tModel.data_scaler = None

tModel.load_data(unroll=False, verbose=1,cross_validation_ratio=0.05) #,cross_validation_ratio=0.05
tModel.print_data()

Loading data for dataset 1 with window_size of 30, stride of 1 and maxRUL of 125. Cros-Validation ratio 0.05
Loading data from file and computing dataframes
cv number
[73, 20, 44, 82, 76]
Printing shapes

Training data (X, y)
(16813, 30, 25)
(16813, 1)
Cross-Validation data (X, y)
(5, 30, 25)
(5, 1)
Testing data (X, y)
(100, 30, 25)
(100, 1)
Printing first 5 elements

Training data (X, y)
[[[0.45977011 0.16666667 0.         ... 0.         0.71317829 0.7246617 ]
  [0.6091954  0.25       0.         ... 0.         0.66666667 0.73101353]
  [0.25287356 0.75       0.         ... 0.         0.62790698 0.62137531]
  ...
  [0.36206897 0.91666667 0.         ... 0.         0.6744186  0.53838719]
  [0.56896552 0.41666667 0.         ... 0.         0.6124031  0.64277824]
  [0.37356322 0.5        0.         ... 0.         0.70542636 0.71361502]]

 [[0.6091954  0.25       0.         ... 0.         0.66666667 0.73101353]
  [0.25287356 0.75       0.         ... 0.         0.62790698 0.62137531]
  [0.540

/home/controlslab/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


# Train the model and test some Tunable Model functionalities

In [9]:
nFeatures = len(selected_features)

lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)

shape = (tModel.data_handler.sequence_length, nFeatures)
model = get_compiled_model(shape, model_type='lstm')
tModel.change_model('ModelRUL_RNN_1', model, 'keras')

tModel.epochs = 100

now = datetime.now()
tensorboard = TrainValTensorBoard(log_dir="./logs/"+now.strftime("%Y%m%d-%H%M%S")+ "/",write_graph=False)



history = tModel.train_model(verbose=1,learningRate_scheduler = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min'))

print(history.history.keys())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 30, 100)           50400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 100)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
_________________________________________________________________
activation_1 (Activation)    (None, 1)                 0         
Total params: 80,651
Trainable params: 80,651
Non-trainable params: 0
_________________________________________________________________
None
t

KeyboardInterrupt: 

In [ ]:
# file = open("results/LSTM/cvtest.csv", "w")
# csvfile = csv.writer(file,lineterminator='\n')

tModel.evaluate_model(['rhs', 'rmse'], round=2, cross_validation=True)
print("scores")

cScores = tModel.scores
#rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.train_time

i = range(1,len(tModel.y_crossVal)+1)


for i, rul_prediction, rul_prediction_rounded, true_rul in zip(i, np.ravel(tModel.y_predicted), tModel.y_predicted_rounded, np.ravel(tModel.y_crossVal)):
    print('{}, {}'.format(rul_prediction_rounded, true_rul))
#     p = "Engine {},Predicted RUL {},Rounded RUL {},Real RUL {}".format(i, rul_prediction, 
#                                                                     rul_prediction_rounded, 
#                                                                     true_rul)
#     print(p)
#     csvfile.writerow(["Engine {} Predicted RUL {} Rounded RUL {} Real RUL {}".format(i,rul_prediction, 
#                                                                     rul_prediction_rounded, 
#                                                                     true_rul)])
print(cScores)
#print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

# csvfile.writerow(["RMSE2: {}".format(rmse2)])

# csvfile.writerow(["RHS: {}".format(rhs)])

# csvfile.writerow(["Time : {} seconds".format(time)])



In [ ]:
tModel.evaluate_model(['rhs', 'rmse'], round=2)
print("scores")

cScores = tModel.scores
#rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.train_time

i = range(1,len(tModel.y_test)+1)

#print(tModel.y_predicted)
#print(tModel.y_predicted_rounded)
#print(tModel.y_test)

for i, rul_prediction, rul_prediction_rounded, true_rul in zip(i, np.ravel(tModel.y_predicted), tModel.y_predicted_rounded, np.ravel(tModel.y_test)):
    print('{}, {}'.format(rul_prediction_rounded, true_rul))
    #print('Engine {}, Predicted RUL {}, Rounded RUL {}, Real RUL {}'.format(i, rul_prediction, 
#                                                                     rul_prediction_rounded, 
#                                                                     true_rul))
    #csvfile.writerow(["Engine {} Predicted RUL {} Rounded RUL {} Real RUL {}".format(i,rul_prediction, 
#                                                                     rul_prediction_rounded, 
#                                                                     true_rul)])
    


print(cScores)
#print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

# csvfile.writerow(["RMSE2: {}".format(rmse2)])

# csvfile.writerow(["RHS: {}".format(rhs)])

# csvfile.writerow(["Time : {} seconds".format(time)])

# file.close()